In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

In [4]:
era = xr.open_dataset('/bsuhome/zacharykeskinen/scratch/era5/ERA5_2020-01-03T20:00_-116.0_45.0_-115.0_46.0.nc')

In [5]:
def convert_pressure_to_pascals(dataset: xr.Dataset) -> xr.Dataset:

    if era['level'].attrs['units'] != 'pascals':
        dataset['level'] = dataset['level'] * 100
        dataset['level'].attrs['units'] = 'pascals'

    else:
        print("Pressure levels already converted to pascals")
    return dataset

In [6]:
def get_vapour_partial_pressure(dataset: xr.Dataset):
    Rv = 461.495
    Rd = 287.05
    alpha = Rv/Rd
    if dataset['q'].attrs['standard_name'] == 'specific_humidity':
        # specific humidity calculation of partial pressure of water vapour
        dataset['vpr'] = dataset['q']*dataset['level']*alpha/(1+(alpha - 1)*dataset['q'])

    return dataset

In [7]:
era = convert_pressure_to_pascals(era)
era = get_vapour_partial_pressure(era)

In [8]:
era

<xarray.Dataset>
Dimensions:    (longitude: 5, latitude: 5, level: 37, time: 1)
Coordinates:
  * longitude  (longitude) float32 -116.0 -115.8 -115.5 -115.2 -115.0
  * latitude   (latitude) float32 46.0 45.75 45.5 45.25 45.0
  * level      (level) int32 100 200 300 500 700 ... 92500 95000 97500 100000
  * time       (time) datetime64[ns] 2020-01-03T20:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 ...
    t          (time, level, latitude, longitude) float32 ...
    q          (time, level, latitude, longitude) float32 3.755e-06 ... 0.003317
    vpr        (time, level, latitude, longitude) float64 0.0006036 ... 532.2
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-04 17:03:14 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...